In [2]:
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from docx import Document

/home/biomega/.cache/pypoetry/virtualenvs/rag-system-on-local-machine-J86ac31T-py3.12/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
doc = Document('./data/diplom.docx')
fullText = []
for para in doc.paragraphs:
    if len(para.text)>0:
        fullText.append(para.text)
print(fullText[232])

Тепловой баланс плазменной печи.


In [6]:
# Для плучения эмбедингов используется модель multilingual-e5-small
# Длина контекстного окна у модели 512 токенов, это нужно учесть
model = SentenceTransformer('intfloat/multilingual-e5-small')
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')

In [12]:
corpus = {}
inter = 0
chunk = ''
for para in fullText:
    if len(tokenizer.tokenize(para))>512:
        print('Fail')
        break
    elif (len(tokenizer.tokenize(chunk))+len(tokenizer.tokenize(para)))<=512:
        chunk = chunk + ' ' + para
        continue
    elif (len(chunk)+len(para))>512:
        corpus[inter]=chunk
        inter+=1
        chunk = para
print(f'Итого исходный текст разбился на {len(corpus.keys())} кусков, где каждый не более 512 токенов')        

Итого исходный текст разбился на 52 кусков, где каждый не более 512 токенов


In [13]:
embeds = {}
for key in corpus.keys():
    embeds[key] = model.encode(corpus[key])

In [14]:
with open('./data/corpus.pickle', 'wb') as handle:
    pickle.dump(corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('./data/embeds.pickle', 'wb') as handle:
    pickle.dump(embeds, handle, protocol=pickle.HIGHEST_PROTOCOL)